# Generate Testing Subsets

To test the performance of the proposed methods at different sizes of the population of devices, we adopt the following strategy: for each target population size $p = 1..P − 1$, where $P$ is the maximum number of devices in the original dataset, we produce $d = 10$ different subsets by selecting p devices at random.

## Libraries and Configurations

Import configuration files

In [61]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [62]:
import pandas as pd

Import **other libraries**

In [63]:
from rich.progress import Progress
from rich import traceback

traceback.install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10d748ed0>>

Custom helper scripts

In [64]:
%cd ..
from scripts import plotHelper, encodingHelper, printHelper
%cd data_exploration_cleaning

/Users/bacci/Library/CloudStorage/SynologyDrive-giovanni/Research 🌱/Repositories/COMPACT/notebooks
/Users/bacci/Library/CloudStorage/SynologyDrive-giovanni/Research 🌱/Repositories/COMPACT/notebooks/data_exploration_cleaning


## Import Data

In [65]:
# Combined dataframe
balanced_df_csv = config["DEFAULT"]["interim_path"] + "balanced_df.csv"

In [66]:
df = pd.read_csv(balanced_df_csv, index_col=0)
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

## Generate Subsets

In [67]:
unique_labels = df["Label"].unique()

In [68]:
for label in unique_labels:
    print(label)

iPhone12Pro_C
S21Ultra_M
iPhone11_M
iPhoneXR_A
iPhone7_F
iPhone12_M
OppoFindX3Neo_A
iPhone11_F
iPhoneXR_L
iPhone11_B
iPhone11_C
OnePlusNord_O
HuaweiP10_Q
iPhoneXR_U
GooglePixel3A_V
XiaomiRedmiNote9S_T
SamsungJ6_K
iPhone7_X
XiaomiA2_E
SamsungM31_A
iPhone12_W
GooglePixel3A_L
SamsungS7_I
HuaweiL21_D
iPhoneXSMax_M
iPhone6_N
SamsungS6_H
HuaweiHonor9_R
HuaweiP20_G
SamsungS4_C
XiaomiRedmi4_B
XiaomiRedmi5_J
XiaomiRedmiNote7_S


In [69]:
from itertools import combinations

import random

random.seed(42)

combs_3 = [random.sample(list(unique_labels), 3) for _ in range(10)]